# visualizing synthetic landscape + SGD

## imports

### mount drive


In [1]:
COLAB = False
if COLAB == True:
    drive.mount('/content/drive')

### pip installs

In [2]:
if COLAB==True:
    !pip install scanpy moscot ott-jax diffrax napari anndata -q
    !pip install "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html -q

### imports

In [3]:
import math, napari, numpy as np
import pandas as pd

from scipy.optimize import minimize
from scipy.ndimage import minimum_filter

import importlib

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

import sys, pathlib

if COLAB == True:
    fh_hmot = "/content/drive/Othercomputers/numac/GitHub/HM-OT" # filehandle for HM-OT
    project_root = pathlib.Path(fh_hmot)
    sys.path.insert(0, fh_hmot)
    sys.path.insert(0, str(project_root / "src"))
else: 
    fh_hmot = '../../'
    project_root = pathlib.Path(fh_hmot)
    sys.path.insert(0, fh_hmot)

from src.utils.waddington import viewer, visual, minima, simulation, differentiation, metrics
from src.utils.waddington import seed_everything
from src.platting import color_utils, palette_utils, string_utils, spatial_scatter, diffmap

## set seed

In [4]:
# Set global seed for reproducibility
GLOBAL_SEED = 42
# np.random.seed(GLOBAL_SEED)
seed = seed_everything(GLOBAL_SEED)

🌱 Setting global seed to 42
  ⚠ PyTorch not available
  ⚠ JAX not available
🌱 All available RNGs seeded.


In [5]:
str_to_color_dict = {
    'U': (1.0, 1.0, 0.0, 1.0),
    'A240': (0.10980392156862745, 0.9019607843137255, 1.0, 1.0),
    'A0': (1.0, 0.20392156862745098, 1.0, 1.0),
    'A120': (1.0, 0.2901960784313726, 0.27450980392156865, 1.0),
    'B210': (0.0, 0.5372549019607843, 0.2549019607843137, 1.0),
    'B150': (0.0, 0.43529411764705883, 0.6509803921568628, 1.0),
    'B330': (0.6392156862745098, 0.0, 0.34901960784313724, 1.0),
    'B30': (1.0, 0.8588235294117647, 0.8980392156862745, 1.0),
    'B270': (0.47843137254901963, 0.28627450980392155, 0.0, 1.0),
    'B90': (0.0, 0.0, 0.6509803921568628, 1.0),
    'C210': (0.38823529411764707, 1.0, 0.6745098039215687, 1.0),
    'C330': (0.7176470588235294, 0.592156862745098, 0.3843137254901961, 1.0),
    'C30': (0.0, 0.30196078431372547, 0.2627450980392157, 1.0),
    'C150': (0.5607843137254902, 0.6901960784313725, 1.0, 1.0),
    'C270': (0.6, 0.49019607843137253, 0.5294117647058824, 1.0),
    'C90': (0.35294117647058826, 0.0, 0.027450980392156862, 1.0)
}

## set parameters (`TIMEPOINTS`, `ASSIGNMENT_RADIUS`, `LANGEVIN` parameters)

In [6]:
%%time
# timepoint indices of snapshots of Langevin dynamics
TIMEPOINTS = [19, 89, 2500] # snapshots taken at these timepoints

# assignment radius (determines cell type assignments):
ASSIGNMENT_RADIUS = 1.6

# Langevin dynamics parameteters
LANGEVIN_n_particles = 100 # number of partciles
LANGEVIN_N_timesteps = 4000 # number of time steps
LANGEVIN_dt = 0.2
LANGEVIN_D = 0.014


SHOW_DYN = True # Set to True to visualize dynamics with napari
# ------------------------------------------------------------------
if __name__ == "__main__" and SHOW_DYN:
    try:
        viewer.launch_viewer_with_differentiation(
            timepoints=TIMEPOINTS,
            langevin_n=LANGEVIN_n_particles,
            langevin_N=LANGEVIN_N_timesteps,
            langevin_dt=LANGEVIN_dt,
            langevin_D=LANGEVIN_D,
            assignment_radius=ASSIGNMENT_RADIUS,
            str_to_color_dict=str_to_color_dict,
        )
        napari.run()
    except Exception as e:
        print("Napari error:", e)
        print("Install with: pip install 'napari[all]' scipy")
elif SHOW_DYN==False:
    img = Image.open(fh_hmot + '/experiments/2.png')
    img

Finding local minima …
→ 15 minima found
Building surface …
Running Langevin simulation …
Constructing differentiation map …
Play ▶︎ to animate.
CPU times: user 26.2 s, sys: 1.32 s, total: 27.6 s
Wall time: 29.1 s
